# Primitives examples

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.1.2
qiskit-ibm-runtime~=0.43.1
```
</details>

The examples in this section illustrate some common ways to use primitives. Before running these examples, follow the instructions in [Install and set up.](install-qiskit)

<Admonition type="note">
    These examples all use the primitives from Qiskit Runtime, but you could use the base primitives instead.
</Admonition>

## Estimator examples

Efficiently calculate and interpret expectation values of the quantum operators required for many algorithms with Estimator. Explore uses in molecular modeling, machine learning, and complex optimization problems.

### Run a single experiment

Use Estimator to determine the expectation value of a single circuit-observable pair.

In [1]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
observable = SparsePauliOp("Z" * 50)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

estimator = Estimator(mode=backend)
job = estimator.run([(isa_circuit, isa_observable)])
result = job.result()

print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

 > Expectation value: 0.5176470588235295
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


### Run multiple experiments in a single job

Use Estimator to determine the expectation values of multiple circuit-observable pairs.

In [2]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]

pubs = []
circuits = [iqp(mat) for mat in mats]
observables = [
    SparsePauliOp("X" * 50),
    SparsePauliOp("Y" * 50),
    SparsePauliOp("Z" * 50),
]

# Get ISA circuits
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

for qc, obs in zip(circuits, observables):
    isa_circuit = pm.run(qc)
    isa_obs = obs.apply_layout(isa_circuit.layout)
    pubs.append((isa_circuit, isa_obs))

estimator = Estimator(backend)
job = estimator.run(pubs)
job_result = job.result()

for idx in range(len(pubs)):
    pub_result = job_result[idx]
    print(f">>> Expectation values for PUB {idx}: {pub_result.data.evs}")
    print(f">>> Standard errors for PUB {idx}: {pub_result.data.stds}")

>>> Expectation values for PUB 0: -0.00146484375
>>> Standard errors for PUB 0: 0.014056384558264724
>>> Expectation values for PUB 1: 0.021484375
>>> Standard errors for PUB 1: 0.015998198169341674
>>> Expectation values for PUB 2: 0.0068359375
>>> Standard errors for PUB 2: 0.013100823433308964


### Run parameterized circuits

Use Estimator to run three experiments in a single job, leveraging parameter values to increase circuit reusability.

In [3]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Step 1: Map classical inputs to a quantum problem
theta = Parameter("θ")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)

number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
individual_phases = [[ph] for ph in phases]

ZZ = SparsePauliOp.from_list([("ZZ", 1)])
ZX = SparsePauliOp.from_list([("ZX", 1)])
XZ = SparsePauliOp.from_list([("XZ", 1)])
XX = SparsePauliOp.from_list([("XX", 1)])
ops = [ZZ, ZX, XZ, XX]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
chsh_isa_circuit = pm.run(chsh_circuit)
isa_observables = [
    operator.apply_layout(chsh_isa_circuit.layout) for operator in ops
]

# Step 3: Execute using Qiskit primitives.

# Reshape observable array for broadcasting
reshaped_ops = np.fromiter(isa_observables, dtype=object)
reshaped_ops = reshaped_ops.reshape((4, 1))

estimator = Estimator(backend, options={"default_shots": int(1e4)})
job = estimator.run([(chsh_isa_circuit, reshaped_ops, individual_phases)])
# Get results for the first (and only) PUB
pub_result = job.result()[0]
print(f">>> Expectation values: {pub_result.data.evs}")
print(f">>> Standard errors: {pub_result.data.stds}")
print(f">>> Metadata: {pub_result.metadata}")

>>> Expectation values: [[ 1.05516412e+00  1.01765759e+00  8.50929350e-01  6.04792761e-01
   3.24372862e-01 -2.19764812e-02 -3.36972711e-01 -6.20029788e-01
  -8.70854693e-01 -9.93336948e-01 -1.06981510e+00 -9.87183534e-01
  -8.38915541e-01 -6.04792761e-01 -3.02982420e-01  2.54927181e-02
   3.45177264e-01  6.10653156e-01  8.56203706e-01  9.93336948e-01
   1.06747094e+00]
 [ 3.01810341e-02  3.80632654e-01  6.45229487e-01  8.78473207e-01
   1.02820630e+00  1.04549446e+00  9.96267146e-01  8.41552719e-01
   6.09774097e-01  2.78954801e-01 -2.78368761e-02 -3.71549041e-01
  -6.60466514e-01 -8.85212661e-01 -1.01677853e+00 -1.05662921e+00
  -1.00974605e+00 -8.43603857e-01 -6.19150729e-01 -2.68992129e-01
   2.16834614e-02]
 [-2.63717774e-03 -3.19098506e-01 -6.19736769e-01 -8.61771081e-01
  -9.92457889e-01 -1.06219659e+00 -9.92457889e-01 -8.37743462e-01
  -5.97760288e-01 -3.07084697e-01  4.98133573e-03  3.14703210e-01
   5.99518406e-01  8.24850593e-01  9.76341803e-01  1.04666654e+00
   9.79565020e

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.


<Admonition type="caution">
The following code block will return an error for Open Plan users because it uses sessions. Open Plan workloads can run only in [job mode](/docs/guides/execution-modes#job-mode) or [batch mode](/docs/guides/execution-modes#batch-mode).
</Admonition>

In [4]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Session,
    EstimatorV2 as Estimator,
)

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = iqp(mat)
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = iqp(mat)
observable = SparsePauliOp("X" * 50)
another_observable = SparsePauliOp("Y" * 50)

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
another_isa_observable = another_observable.apply_layout(
    another_isa_circuit.layout
)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)

    estimator.options.resilience_level = 1

    job = estimator.run([(isa_circuit, isa_observable)])
    another_job = estimator.run(
        [(another_isa_circuit, another_isa_observable)]
    )
    result = job.result()
    another_result = another_job.result()

    # first job
    print(f" > Expectation value: {result[0].data.evs}")
    print(f" > Metadata: {result[0].metadata}")

    # second job
    print(f" > Another Expectation value: {another_result[0].data.evs}")
    print(f" > More Metadata: {another_result[0].metadata}")

 > Expectation value: -0.01611328125
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}
 > Another Expectation value: 0.03125
 > More Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


<span id="sampler-examples"></span>
## Sampler examples

Generate entire error-mitigated quasi-probability distributions sampled from quantum circuit outputs. Leverage Sampler’s capabilities for search and classification algorithms like Grover’s and QVSM.

### Run a single experiment

Use Sampler to return the measurement outcome as bitstrings or counts of a single circuit.

In [5]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

sampler = Sampler(backend)
job = sampler.run([isa_circuit])
result = job.result()

# Get results for the first (and only) PUB
pub_result = result[0]

print(f" > First ten results: {pub_result.data.meas.get_bitstrings()[:10]}")

 > First ten results: ['0011101110101111000010100101100001011100101100110101110010111011010010011101101100011101000000001000000010001010110001000110001', '0101101001011100100011010010101111101010100100011001100001011011000000001000111011100010110000100110000111100100100000010111000', '1100000001010110100010011010001111001101100100101010011110101100100001001000000001010001111000100000100000011000110000100111110', '1011100001010110000101000100000000000101010111101000111110001000010001010011101001100001000110100000000000001100010010111000001', '1001101001111011001110101111000100001000100001110011100001111000000010100110010101100000010010100110000010010000010001111000100', '1000111000010111001100100111100101001011110001011101111000110111101010010001110011011010000001000111010011101101000011010100000', '0010000111100111000111101100001010000011100000110000001101001010001000000001000100000001001100110110000110000001001010011000101', '10111011110110001011111001111011011000000111110011101010111

### Run multiple experiments in a single job

Use Sampler to return the measurement outcome as bitstrings or counts of multiple circuits in one job.

In [6]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]
circuits = [iqp(mat) for mat in mats]
for circuit in circuits:
    circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuits = pm.run(circuits)

sampler = Sampler(mode=backend)
job = sampler.run(isa_circuits)
result = job.result()

for idx, pub_result in enumerate(result):
    print(
        f" > First ten results for pub {idx}: {pub_result.data.meas.get_bitstrings()[:10]}"
    )

 > First ten results for pub 0: ['1010000010011111010100110000111100111011000001001011000010000011001110110010001110100100010100100101000101000100000000110101001', '0101111100011101010000001100110111010001001100000010000101101000100010000011110111010100001101001110000010100010001011011101000', '1101110100011000101111001001001011000101100110100110101110100110011011010000110000111000010001111000110001100100100000010000100', '0100011001000001110001110110011001011101100011100000000010000010110111000100011010100100000101111100000111100100001010000100000', '0000001010101001001111101110110101001111011001000101101110001000000011011010000000010100000100000000010100001000000010010100100', '0111110110111100000100010101010010001010100100100010000000100100011010110010111000001101001000001100000001100000000100010000110', '1111110101001010111000001100010000101011110101010000011101001010100011111100001110100010001101000100100000100101001100000000000', '0110010111000000001000101100000110001000100101001

### Run parameterized circuits

Run several experiments in a single job, leveraging parameter values to increase circuit reusability.

In [7]:
import numpy as np
from qiskit.circuit.library import real_amplitudes
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

# Step 1: Map classical inputs to a quantum problem
circuit = real_amplitudes(num_qubits=n_qubits, reps=2)
circuit.measure_all()

# Define three sets of parameters for the circuit
rng = np.random.default_rng(1234)
parameter_values = [
    rng.uniform(-np.pi, np.pi, size=circuit.num_parameters) for _ in range(3)
]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

# Step 3: Execute using Qiskit primitives.
sampler = Sampler(backend)
job = sampler.run([(isa_circuit, parameter_values)])
result = job.result()
# Get results for the first (and only) PUB
pub_result = result[0]
# Get counts from the classical register "meas".
print(
    f" >> First ten results for the meas output register: {pub_result.data.meas.get_bitstrings()[:10]}"
)

 >> First ten results for the meas output register: ['1001010010001101010111001101110000100000100011001010101000110100100100001111110000000100001010100000101110100011100000100000100', '1101011000001000010100101011000000110110010000100000010111100001010100100011110111010011010100100010101000100100100011101000010', '0101000011100100110110110101100001100001100100001001110000011101100011001110000010011111100010110001100100011010101000011101100', '1001011100101000100011100100010111000111000001011001011000101111111100101111110111110011000000000000000101100010011110101000010', '0011110100010100011000101001110001110100011101010000101010010111001010110010010101100011100101110101010100100011111001101000111', '0001000100111010001000100100010000000100100001011100001010101100101010001001001100001100100100001010011011101100011001100100010', '1110010011110001011000111001100001110001100000111011111011111001011001101011001100011000001100100000001111010110001111010001101', '00001101010000101111010011110

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.


<Admonition type="caution">
The following code block will return an error for users on the Open Plan, because it uses sessions. Workloads on the Open Plan can run only in [job mode](/docs/guides/execution-modes#job-mode) or [batch mode](/docs/guides/execution-modes#batch-mode).
</Admonition>

In [8]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.quantum_info import random_hermitian
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = iqp(mat)
circuit.measure_all()
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = iqp(mat)
another_circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run([isa_circuit])
    another_job = sampler.run([another_isa_circuit])
    result = job.result()
    another_result = another_job.result()

# first job

print(
    f" > The first ten measurement results of job 1: {result[0].data.meas.get_bitstrings()[:10]}"
)

 > The first ten measurement results of job 1: ['0100010101010011110001100011010000101111100011100000000111100000111101000000000101001001000000010100000011100101000000101011100', '1101010100000010100001100100010100110011010100010101010100111100001000000100000100010100010010000001001011000001000000000000000', '0101100010000110000100010000011001100001100001010110010010010101101001001001110000011011101011000100010001100001001100101000010', '0111001101110101001011011100100000010111100011111100010110111000001101001100001101011000010000010011100010001000000010000100110', '1100001001001001001110000010011100011000010100000011100101101000010000010001001000011010000001010010110101100100000001000000100', '1111010111100111100001000011000001000011010101011111011100000100100100100010111000000000001101001000000100000100100100000010010', '1111110011111000001100011000010000100100111011011000011010101011001100110001010110000110101111011001001011000010000000001000000', '0000011011000001011000001110001110

In [9]:
# second job
print(
    " > The first ten measurement results of job 2:",
    another_result[0].data.meas.get_bitstrings()[:10],
)

 > The first ten measurement results of job 2: ['0000011000010101110010010100011110011010000000010011110101100000110101000101101100000100001000001000111110000100111001100011000', '1101000101010010001010011101011001010010101011100110000111100000110111100000010000000100000011011000000000000001001011001000000', '1001101101101010000101111110011110011010001010011000000110010010111000101001011010011010101010010110111010001011000001000100000', '1011110001111110001100001010011111001000100010100000000101000000111011000011000000110001010010001011111001000001010010100010100', '0100001010110000111011111000010110010101000110011000010000110100001000000001001010011010010000000010010100110010110100001000100', '1010000101100000100101001010000010000010001000101010001100100100011010100010000000010001000100001100010010000001110000000101000', '1111101010011110011111010110001110011111011100000000001100110000000101000011011001000110110000110110001000110010011010100110000', '1000000011000101111001011010011001

## Next steps

<Admonition type="tip" title="Recommendations">
    - [Specify advanced runtime options.](runtime-options-overview)
    - Practice with primitives by working through the [Cost function lesson](/learning/courses/variational-algorithm-design/cost-functions) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile/) section.
    - Try the [Compare transpiler settings](/docs/tutorials/circuit-transpilation-settings) tutorial.
    - Read [Migrate to V2 primitives](/docs/guides/v2-primitives).
    - Understand the [Job limits](/docs/guides/job-limits) when sending a job to an IBM&reg; QPU.
</Admonition>